In [3]:
# https://blog.csdn.net/dyg66/article/details/105835831

from bs4 import BeautifulSoup
import re
import urllib.request, urllib.error
import time
import pandas as pd

year = ['11', '12', '13', '14', '15', '16', '17', '18', '19']
month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
url_failed = []

# get_data用到的正则表达式
find_date = re.compile(r'<div>(.*?)</div>')
find_high = re.compile(r'<div style="width: 100px">(\d*)</div>')
find_wind = re.compile(r'<div style="width:200px;">(.*?)</div>')

# get_cty用到的正则表达式
find_city = re.compile(r'<a href="(.*?)/index.html"')
find_name = re.compile(r'历史天气">(.*?)</a>')

# 基本思路
def main():
    # 获取城市名拼音，用于拼接网页url；拼音在列表的偶数位；[2:4:1]用于测试，去掉则可以获取全部城市的数据
    city = get_city()[0::2][2:4:1]  
    datalist = getData(city)  # 获取天气数据
    saveData(datalist)  # 保存数据
    print(url_failed)  # 打印访问失败页面的列表

# 获取城市名
def get_city():
    city_list = []
    url = 'http://lishi.tianqi.com/'
    html = askURL(url)
    soup = BeautifulSoup(html, "html.parser")
    for item in soup.find_all('ul', class_="table_list"):
        # print(item)  # 测试：查看item的全部信息，接下来获取想要的信息
        data = []
        for each_city in item.find_all('li'):
            # print(each_city)
            # print('-'*50)
            city_info = str(each_city)
            one_city = re.findall(find_city, city_info)
            one_name = re.findall(find_name, city_info)
            if len(one_city) > 0:
                data.extend(one_city)
                data.extend(one_name)
        city_list.extend(data)
    # print(city_list[0::2][0:9:1])
    # print(len(city_list))
    return city_list

# 获取天气数据
def getData(city):
    datalist = []
    for c in city:
        data_city = []
        for y in year:
            for m in month:
                url = 'http://lishi.tianqi.com/' + c + '/20' + y + m + '.html'
                html = askURL(url)
                # 逐一解析
                soup = BeautifulSoup(html, "html.parser")
                for item in soup.find_all('ul', class_="lishitable_content clearfix"):
                    if len(item) > 0:
                        for each_date in item.find_all('li'):
                            # print(each_date)
                            # print('-'*50)
                            data = []  # 保存一个城市的所有天气信息
                            info = str(each_date)
                            date = re.findall(find_date, info)
                            high_temp = re.findall(find_high, info)
                            wind = re.findall(find_wind, info)
                            # print(date)
                            data.append(c)
                            if len(high_temp) > 0 and len(date) > 0:
                                # print(date[0])
                                if len(date[0]) == 10:
                                    data.append(date[0])
                                elif len(date[0]) > 10:
                                    date_1 = str(date[0]).split('>')[1].split('<')[0]
                                    # print(date_1)
                                    data.append(date_1)
                                data.append(high_temp[0])
                                data.append(date[1])
                                data.append(date[2])
                                data.append(wind[0])
                            # print(data)
                            data_city.append(data)
                        print('Finish ' + c + '/20' + y + m + '-' * 30)
        # data_city.insert(0, c)
        # print(data_city)
        print('Get the data of ' + c)
     #的   datalist.append(data_city)
        # print(datalist)
        # print(len(datalist))
    for i in range(0, len(datalist)):
        for k in datalist[i]:
            # print(len(k))
            if len(k) != 6:
                datalist[i].remove(k)
    # print(datalist)
    return datalist


# 得到指定url的网页内容
def askURL(url):
    # 模拟浏览器头部信息，向服务器发送信息，告诉服务器我们可以接受什么水平的内容; 注意键值对的键一定要写对
    head = {
        "User-Agent": "填入自己的header中的信息"}
    try:
        request = urllib.request.Request(url, headers=head)
        response = urllib.request.urlopen(request, timeout=20)
        html = response.read().decode('utf-8')
        return html
    except:
        time.sleep(1)
        try:  # 有时候网页会卡顿，所以可以尝试再进一次
            request = urllib.request.Request(url, headers=head)
            response = urllib.request.urlopen(request, timeout=20)
            html = response.read().decode('utf-8')
            return html
        except:  # 如果再次尝试还是进不去，那就显示“访问超时”
            print('该页 %s 访问超时' % url)
            url_failed.append(url)  # 保存访问失败的页面，用于后续数据处理
            time.sleep(1)
            request = urllib.request.Request('http://lishi.tianqi.com/changsha/201201.html', headers=head)
            # 无法访问该页面时，访问一个其他页面，确保循环访问顺利进行 （这里的方法是自己想的，应该有更先进的方法来跳过访问失败的页面
            response = urllib.request.urlopen(request, timeout=20)
            html = response.read().decode('utf-8')
            return html

# 保存数据到csv
def saveData(datalist):
    print("save...")
    data = []
    for k in range(0, 6):
        var = []
        for i in range(0, len(datalist)):
            for j in range(0, len(datalist[i])):
                if len(datalist[i][j]) == 6:
                    var.append(datalist[i][j][k])
        data.append(var)
    df = pd.DataFrame(
        {'City': data[0], 'Date': data[1], 'HighT': data[2], 'LowT': data[3], 'Weather': data[4], 'Wind': data[5]})
    df.to_csv('city_weather.csv', index=False, sep=',')


if __name__ == "__main__":
    # 调用函数
    main()
    print('爬取完毕')

该页 http://lishi.tianqi.com/ 访问超时


UnicodeEncodeError: 'latin-1' codec can't encode characters in position 0-4: ordinal not in range(256)